Changing from regression to classification:

- change last activation to Sigmoid to get a value between 0 and 1
- use as many output nodes as you have choices (2 choices = 2 nodes)
- use CrossEntropyLoss to better measure error for classification

We are using "logistic regression" for classification tasks because the logistic function (aka, sigmoid) looks like a sideways "S" and ranges between 0 and 1. It essentially produces a "percent likelihood" that the data is in class A or B, etc.

In [ ]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import random
import sklearn.metrics

In [2]:
adultdf = pd.read_csv("/home/jeckroth/csci431-public/datasets/uci/adult/adult.data", header=None)
adultdf.columns = ["age", "workclass", "fnlwgt", "education", "educationNum",
                  "maritalStatus", "occupation", "relationship", "race",
                  "sex", "capitalGain", "capitalLoss", "hoursPerWeek",
                  "nativeCountry", "income"]
adultdf

,age,workclass,fnlwgt,education,educationNum,maritalStatus,occupation,relationship,race,sex,capitalGain,capitalLoss,hoursPerWeek,nativeCountry,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


Need to turn some of these categorical columns into "one-hot" columns, such as workclass: Private = {0,0,0}, Self-emp-not-inc = {0,1,0}, etc. This way, the neural net does not see labels like Private = 1, Self-emp-not-inc = 2, etc. which may lead it to believe Private is "close to" Self-emp-not-inc, etc.

In [3]:
# example of pandas' get_dummies on a single column:
pd.get_dummies(adultdf[["workclass"]], prefix=["workclass"], columns=["workclass"])

,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay
0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
32556,0,0,0,0,1,0,0,0,0
32557,0,0,0,0,1,0,0,0,0
32558,0,0,0,0,1,0,0,0,0
32559,0,0,0,0,1,0,0,0,0


In [4]:
onehot_columns = ["relationship"]
x = pd.get_dummies(adultdf[onehot_columns],
                    prefix=onehot_columns,
                    columns=onehot_columns)
x

,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife
0,0,1,0,0,0,0
1,1,0,0,0,0,0
2,0,1,0,0,0,0
3,1,0,0,0,0,0
4,0,0,0,0,0,1
...,...,...,...,...,...,...
32556,0,0,0,0,0,1
32557,1,0,0,0,0,0
32558,0,0,0,0,1,0
32559,0,0,0,1,0,0


Because we are doing classification, we will use CrossEntropyLoss for our criterion function. The documentation shows that the "input" (the last layer of the network) should have 2 neurons because we have two classes (<=50K, >50K) for our target. The docs also show we need to make our y target values a single number that is 0 or 1 (since we have two classes). We do not use "onehot" encoding on these y values.

In [5]:
y = pd.Series([x == ' <=50K' for x in list(adultdf["income"])])
y

0         True
1         True
2         True
3         True
4         True
         ...  
32556     True
32557    False
32558     True
32559     True
32560    False
Length: 32561, dtype: bool

**NOTE!!!**

There are only 24% of one class, so our model has to at least get less error than this! It can simply learn to always say "1" or whatever, ignoring the input data, and still be wrong only 24% of the time!

In [6]:
y_true = len(list(filter(lambda x: x, y)))
y_false = len(list(filter(lambda x: not x, y)))
print(y_true, y_false, 1-float(y_true)/float(y_true+y_false))

24720 7841 0.2408095574460244


In [7]:
indexes = pd.Series(y.sample(frac=1.0, random_state=0).index)
train_idxs = indexes.iloc[range(0, int(len(indexes)*0.6))]
val_idxs = indexes.iloc[range(int(len(indexes)*0.6), int(len(indexes)*0.8))]
test_idxs = indexes.iloc[range(int(len(indexes)*0.8), len(indexes))]
train_x = x.iloc[train_idxs]
val_x = x.iloc[val_idxs]
test_x = x.iloc[test_idxs]
train_y = y.iloc[train_idxs]
val_y = y.iloc[val_idxs]
test_y = y.iloc[test_idxs]
train_y

22278     True
8950      True
7838      True
16505     True
19140    False
         ...  
14525     True
26826    False
18552     True
17957    False
27290    False
Length: 19536, dtype: bool

In [8]:
model = torch.nn.Sequential(
    torch.nn.Linear(train_x.shape[1], 100), # compute number of columns from shape
    torch.nn.ReLU(),
    torch.nn.Dropout(p=0.5), # 50% of weights will not be trained each epoch
    torch.nn.Linear(100, 2),
    torch.nn.Sigmoid()
)
model.cuda()

Sequential(
  (0): Linear(in_features=6, out_features=100, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=100, out_features=2, bias=True)
  (4): Sigmoid()
)

In [9]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [10]:
train_x_gpu = torch.tensor(train_x.to_numpy()).float().cuda()
val_x_gpu = torch.tensor(val_x.to_numpy()).float().cuda()
train_y_gpu = torch.tensor(train_y.to_numpy()).long().cuda()
val_y_gpu = torch.tensor(val_y.to_numpy()).long().cuda()
train_x_gpu

tensor([[0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]], device='cuda:0')

In [11]:
for epoch in range(100):
    optimizer.zero_grad()
    train_pred = model(train_x_gpu)
    train_loss = criterion(train_pred, train_y_gpu)
    val_pred = model(val_x_gpu)
    val_loss = criterion(val_pred, val_y_gpu)
    
    # compute % accuracy (actually, error)
    # first, convert the two outputs into a single prediction
    train_pred_label = [0 if pred[0] > pred[1] else 1 for pred in train_pred.cpu().detach().tolist()]
    val_pred_label = [0 if pred[0] > pred[1] else 1 for pred in val_pred.cpu().detach().tolist()]
    train_error = 1.0 - sklearn.metrics.accuracy_score(train_y_gpu.cpu().tolist(), train_pred_label)
    val_error = 1.0 - sklearn.metrics.accuracy_score(val_y_gpu.cpu().tolist(), val_pred_label)
    print(train_loss.item(), val_loss.item(), train_error, val_error)
    train_loss.backward()
    optimizer.step()

0.7003028988838196 0.6999354958534241 0.623413185913186 0.6101044226044225
0.6997714042663574 0.7001855373382568 0.6121519246519247 0.6194717444717445
0.6995231509208679 0.69940185546875 0.607545045045045 0.6067260442260443
0.6989419460296631 0.6991714835166931 0.5998669123669124 0.6030405405405406
0.6982547640800476 0.6979097127914429 0.5875307125307125 0.578931203931204
0.6981114745140076 0.6978065967559814 0.5807739557739557 0.577088452088452
0.6972410082817078 0.6980265378952026 0.5656224406224406 0.5793918918918919
0.6971713304519653 0.6970458030700684 0.5664414414414414 0.5718673218673218
0.6968867182731628 0.6964974999427795 0.5625 0.5571253071253071
0.6962475776672363 0.6964007019996643 0.5534398034398034 0.5528255528255528
0.6956932544708252 0.6957958936691284 0.5377764127764127 0.5426904176904177
0.6955249905586243 0.6951910257339478 0.5341932841932842 0.5304054054054055
0.695032000541687 0.694703221321106 0.524416461916462 0.527487714987715
0.6945065259933472 0.6939409971237